
# Notes
- Prio: Fast testing of models in dev mode
  - [x] connect Jupyter 
    - [ ] Framework 
    
- RigidNeuralPrior can fit as a regular neural prior after some (tested in 500 epochs) iterations
- Transform and refinement layer works, but lacks regularization, that maybe cyclic smoothness provides (todo experiment)
- If NN in refinement has big weight, it deforms quickly
- Add same flow along z-axis as it is in gravity when we know the transformation (todo experiment)
- In this sample, the transformation is fitted correctly
- With Neural Prior, it will never be correct transformation, since Neural Prior can compensate translation with bias? But bias is only 1 number
- Paralel computation of Prior and Transformation? (todo experiment)
- Fit transformation first, then fit (Sequentialy/jointly) neural prior? (todo experiment)
- How to trust rigid transform more? (todo experiment)
- How to find robust transformation

In [1]:
# NN index "distance" transform
    # - construct points around the original point cloud in another dimension (each point has its surroundings a row means index)
    # - distance is coded as a order in which points goes (for example, you add points in one "cell" and those are first, adjacent two cells are seconds)
    # - overlapping points that are "more distant" comes later along the axis, so if you order the final value backwards, the closest should be in the voxel
    # - this can maybe be generalizable into K-NN? Such as using K voxel grids and unique indices?
    # - task for Ruslan?
    # - precision is cell size, because we do not know which one is closer in that cell. Well we know because of values and they can be subtracted, but that might be to much of a hassle. And cell can encode only one value, so we are bounded by this one
    # - max_radius parameter defines the amount of constructed points
    # - you can construct all points and then sort it based on distance to the central points
    # - Ordering of operations are crucial to the solution
    # - Basically you are implementing ball query to voxel grid
%load_ext autoreload
%autoreload 2



import os
os.chdir('/home/vacekpa2/4D-RNSFP')
import torch
import time
import numpy as np
import cupy as cp
from cucim.core.operations import morphology
from pytorch3d.ops.knn import knn_points

from data.dataloader import SFDataset4D
from vis.deprecated_vis import *
from loss.flow import DT
from vis.deprecated_vis import visualize_points3D, visualize_multiple_pcls, imshow



device = torch.device('cuda:0')

dataset = SFDataset4D(dataset_type='waymo', n_frames=1)
data = dataset.__getitem__(80)

device = torch.device('cuda:0')
pc1 = data['pc1'].to(device)
pc2 = data['pc2'].to(device)
id_mask1 = data['id_mask1'].to(device)


os.system('nvidia-smi')


Fri Oct 27 12:20:28 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB           On | 00000000:01:00.0 Off |                    0 |
| N/A   26C    P0               57W / 400W|   3037MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

0

### 1-NN module producing indices and distances

### Input: pc1, pc2
### Output: indices, distances

### Features
- stored dt
- grid factor
- Tensor of indices
- automatic recalc of coordinates
- batch forward pass
- If not necessary: do not generate inside cell NN for precise NN 

### Do not Generate:
- K-NN
- longer time in second iter, if not necessary

### Representation
- Class with forward pass
- indices: (B, N, 1)
- distances: (B, N, 1)


### Pseudo code
- [x] get pc1, pc2 
- [x] construct grid from ranges
- [x] transform pc coordinates to grid coordinates
- [x] compute dt and index grid (original points, not dt indices)
- [x] use dt indices to gather correspondences
- [x] return distances and indices
- [x] refine for speed and readability
- [x] encapsulate to module

In [2]:
# %%timeit -n 10
from tqdm import tqdm
from loss.flow import FastNN, SmoothnessLoss, DT

    
# DT_module = DT(pc1, pc2)
%timeit -n 100 FastNN(pc1,pc2)
SmoothnessModule = SmoothnessLoss(pc1, pc2=None, K=8, pc2_smooth=False, forward_weight=False)
pred_flow = torch.zeros(pc1.shape, device=pc1.device, requires_grad=True)
# dist, NN_indices = NN_module(pc1, pred_flow, pc2)
 
optimizer = torch.optim.Adam([pred_flow], lr=0.001)

for i in tqdm(range(700)):
    
    dist, NN_indices = NN_module(pc1, pred_flow, pc2)
    # dist, NN_indices = DT_module.torch_bilinear_distance(pc1 + pred_flow)
    smooth_loss = SmoothnessModule(pc1, pred_flow, pc2)
    
    loss = dist.mean() + smooth_loss.mean()
    loss.backward()
    # print(loss.item())
    
    optimizer.step()
    optimizer.zero_grad()

243 ms ± 257 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


  0%|          | 0/700 [00:00<?, ?it/s]


NameError: name 'NN_module' is not defined

In [ ]:
import os
os.system('CUDA_LAUNCH_BLOCKING=1')
from data.toy import get_frames
from pytorch3d.ops.knn import knn_points
import torch
import time

gpu = 'cuda:1'
pc1, pc2 = get_frames(gpu=gpu)
device = torch.device(gpu)
# new Fast KNN
meter_range = (100,100,5)
grid_cell = 0.1
volume_size = (int(meter_range[0] / grid_cell) + 2, int(meter_range[1] / grid_cell) + 2, int(meter_range[2] / grid_cell) + 2)
volume = torch.zeros(volume_size, device=device, dtype=torch.long)



N = 1000 * 1000 * 50 * 2 ** 2
pts = torch.rand((N,3), device=device).unsqueeze(0) * 90
pts[:, :, 2] = 1



st = time.time()
dist, nn, _ = knn_points(pts, pc1, K=1)
print(time.time() - st)

cell_indices = (pts[0] / grid_cell).long()
pc1_indices = (pc1[0] / grid_cell).long().clip(0, 49)

print(nn.shape)
st = time.time()
volume[cell_indices[:,0], cell_indices[:,1], cell_indices[:,2]] = nn[0,:,0]
print(time.time() - st)

st = time.time()
pc1_nn = volume[pc1_indices[:,0], pc1_indices[:,1], pc1_indices[:,2]]
print(time.time() - st)